# SQL Setup
This notebook establishes a local SQLite database to store, index, and manage the processed housing affordability dataset. By migrating from raw CSV to a relational database, the project ensures data portability, schema integrity, and optimized query performance for downstream dashboarding.

## Import Libraries

In [2]:
import pandas as pd
import sqlite3

## Load data

In [3]:
df = pd.read_csv("data/housing_funnel_ready.csv")
df.head()

,NAME,median_household_income,median_home_value,state,year,mortgage_rate,hpi,real_income,price_to_income_ratio,monthly_income,annual_tax_ins,monthly_payment_proxy,monthly_piti,total_monthly_debt,total_dti,income_qualified,savings_capable,mortgage_eligible,homeowner_proxy,monthly_tax_ins
0,Alabama,59674,200900,1,2022,5.344038,607.9425,16287.583333,3.366625,4972.833333,3415.3,896.886031,1181.494364,1581.494364,0.318027,1,1,0,0,251.125
1,Alaska,88121,336900,2,2022,5.344038,607.9425,16287.583333,3.823152,7343.416667,5727.3,1504.036355,1981.311355,2381.311355,0.324278,1,0,0,0,421.125
2,Arizona,74568,402800,4,2022,5.344038,607.9425,16287.583333,5.401781,6214.000000,6847.6,1798.236402,2368.869735,2768.869735,0.445586,0,0,0,0,503.500
3,Arkansas,55432,179800,5,2022,5.344038,607.9425,16287.583333,3.243614,4619.333333,3056.6,802.688444,1057.405110,1457.405110,0.315501,1,1,0,0,224.750
4,California,91551,715900,6,2022,5.344038,607.9425,16287.583333,7.819685,7629.250000,12170.3,3196.021450,4210.213117,4610.213117,0.604281,0,0,0,0,894.875


## Create SQLite Database

In [4]:
conn = sqlite3.connect("housing.db")
cursor = conn.cursor()

## Create Table Schema

In [5]:
create_table_query = """
CREATE TABLE IF NOT EXISTS housing_funnel (
    state_name TEXT,
    state_code TEXT,
    year INTEGER,
    median_household_income REAL,
    median_home_value REAL,
    mortgage_rate REAL,
    price_to_income_ratio REAL,
    income_qualified INTEGER,
    savings_capable INTEGER,
    mortgage_eligible INTEGER,
    homeowner_proxy INTEGER
);
"""
cursor.execute(create_table_query)
conn.commit()


## Insert Data into SQL

In [6]:
df.to_sql(
    "housing_funnel",
    conn,
    if_exists="replace",
    index=False
)

52

## SQL Queries

In [ ]:
# Funnel Overview 
query = """
SELECT
    COUNT(*) AS total_households,

    AVG(income_qualified) AS income_qualified_rate,

    AVG(
        CASE WHEN income_qualified = 1 
        THEN savings_capable END
    ) AS savings_capable_rate,

    AVG(
        CASE 
            WHEN income_qualified = 1 
             AND savings_capable = 1
            THEN mortgage_eligible 
        END
    ) AS mortgage_eligible_rate,

    AVG(homeowner_proxy) AS ownership_rate
FROM housing_funnel;
"""

funnel_overview = pd.read_sql(query, conn)
funnel_overview.to_csv("data/funnel_overview.csv", index=False)
funnel_overview

,total_households,income_qualified_rate,savings_capable_rate,mortgage_eligible_rate,ownership_rate
0,52,0.711538,0.513514,0.631579,0.230769


In [8]:
# Funnel by Year
query = """
SELECT
    year,
    AVG(income_qualified) AS income_rate,
    AVG(savings_capable) AS savings_rate,
    AVG(mortgage_eligible) AS mortgage_rate,
    AVG(homeowner_proxy) AS ownership_rate
FROM housing_funnel
GROUP BY year
ORDER BY year;
"""

funnel_by_year = pd.read_sql(query, conn)
funnel_by_year.to_csv("funnel_by_year.csv", index=False)
funnel_by_year

,year,income_rate,savings_rate,mortgage_rate,ownership_rate
0,2022,0.711538,0.365385,0.230769,0.230769


In [ ]:
# Funnel by State
query = """
SELECT
    NAME AS state_name,
    AVG(income_qualified) AS income_rate,
    AVG(savings_capable) AS savings_rate,
    AVG(mortgage_eligible) AS mortgage_eligibility_rate,
    AVG(homeowner_proxy) AS ownership_rate,
    AVG(price_to_income_ratio) AS avg_price_to_income
FROM housing_funnel
GROUP BY state_name
ORDER BY savings_rate ASC;
"""

funnel_by_state = pd.read_sql(query, conn)
funnel_by_state.to_csv("data/funnel_by_state.csv", index=False)
funnel_by_state

,state_name,income_rate,savings_rate,mortgage_eligibility_rate,ownership_rate,avg_price_to_income
0,Alaska,1.0,0.0,0.0,0.0,3.823152
1,Arizona,0.0,0.0,0.0,0.0,5.401781
2,California,0.0,0.0,0.0,0.0,7.819685
3,Colorado,0.0,0.0,0.0,0.0,5.947235
4,Connecticut,1.0,0.0,0.0,0.0,3.926314
5,Delaware,1.0,0.0,0.0,0.0,4.103488
6,District of Columbia,0.0,0.0,0.0,0.0,7.038712
7,Florida,0.0,0.0,0.0,0.0,5.109447
8,Georgia,1.0,0.0,0.0,0.0,4.083090
9,Hawaii,0.0,0.0,0.0,0.0,8.869973


In [ ]:
# Drop-off Diagnostics
query = """
SELECT
    'Income Qualification' AS stage,
    1 - AVG(income_qualified) AS dropoff_rate
FROM housing_funnel

UNION ALL

SELECT
    'Savings Capability',
    1 - AVG(
        CASE WHEN income_qualified = 1 
        THEN savings_capable END
    )
FROM housing_funnel

UNION ALL

SELECT
    'Mortgage Eligibility',
    1 - AVG(
        CASE 
            WHEN income_qualified = 1 
             AND savings_capable = 1
            THEN mortgage_eligible 
        END
    )
FROM housing_funnel;
"""

drop_off = pd.read_sql(query, conn)
drop_off.to_csv("data/drop_off.csv", index=False)
drop_off

,stage,dropoff_rate
0,Income Qualification,0.288462
1,Savings Capability,0.486486
2,Mortgage Eligibility,0.368421


In [ ]:
# State Ranking
query = """
SELECT
    NAME AS state_name,
    AVG(homeowner_proxy) AS ownership_rate,
    AVG(savings_capable) AS savings_rate,
    AVG(price_to_income_ratio) AS price_to_income_ratio
FROM housing_funnel
GROUP BY state_name
ORDER BY ownership_rate ASC;
"""

state_ranking = pd.read_sql(query, conn)
state_ranking.to_csv("data/state_ranking.csv", index=False)
state_ranking

,state_name,ownership_rate,savings_rate,price_to_income_ratio
0,Alabama,0.0,1.0,3.366625
1,Alaska,0.0,0.0,3.823152
2,Arizona,0.0,0.0,5.401781
3,Arkansas,0.0,1.0,3.243614
4,California,0.0,0.0,7.819685
5,Colorado,0.0,0.0,5.947235
6,Connecticut,0.0,0.0,3.926314
7,Delaware,0.0,0.0,4.103488
8,District of Columbia,0.0,0.0,7.038712
9,Florida,0.0,0.0,5.109447
